<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
        <img src="../figs/uoc_masterbrand_3linies_positiu.png", align="left">
    </div>
    <div style="float: right; width: 50%;">
        <p style="margin: 0; padding-top: 22px; text-align:right;">M2.855 · Models avançats de mineria de dades</p>
        <p style="margin: 0; text-align:right;">Màster universitari en Ciència de dades (<i>Data science</i>)</p>
        <p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
    </div>
</div>
<div style="width:100%;">&nbsp;</div>

# Exemple de combinació de classificadors

En aquest exemple compararem l'ús d'arbres de decisió i models _ensemble_, en concret, el model **Random Forest**.


## Introducció

L'_ensemble learning_ és una estratègia en què es fa servir un grup de models per resoldre un problema mitjançant la combinació estratègica de diversos models d'aprenentatge automàtic en un sol model predictiu. 

En general, els mètodes d'ensemble s'utilitzen principalment per millorar la precisió del rendiment general d'un model i per combinar diversos models diferents, també coneguts com a *aprenents bàsics*, per predir els resultats, en lloc d'utilitzar un sol model.

Per què entrenem tants classificadors diferents en lloc d'un de sol? L'ús de diversos models per predir el resultat final en realitat redueix la probabilitat de sospesar les decisions preses per models deficients (sobreentrenats, no degudament ajustats, etc.).

Com més diversos siguin aquests aprenents bàsics, més poderós serà el model final. 

Tinguem en compte que en qualsevol model d'aprenentatge automàtic, l'error de generalització ve donat per la suma de quadrats de biaix + variància + error irreductible. 

Els errors irreductibles són una cosa que va més enllà de nosaltres! No els podem reduir.

Tot i això, utilitzant ensembles podem reduir el biaix (bias) i la variància d'un model. Això redueix l'error de generalització general.

La **compensació de biaix-variància** és el punt de referència més important que diferencia un model robust d'un inferior (entenguem per inferior un model no gaire generalitzable). 

Tot i que no és una regla exacta, en l'aprenentatge automàtic, els models que tenen un biaix alt tendeixen a tenir una variància més baixa i viceversa.

Hem estat parlant de biaix i variància. Però veiem què entenem per un biaix d'un model i per variància d'un model.

1. **Biaix**: el biaix és un error que sorgeix a causa de suposicions falses realitzades en la fase d'aprenentatge d'un model. Un biaix alt pot fer que un algorisme d'aprenentatge ometi informació important i correlacions entre les variables independents i les etiquetes de classe, per la qual cosa no s'ajusta al model.

2. **Variància**: la variància ens diu com de sensible és un model als petits canvis en les dades d'entrenament. És a dir, quant canvia el model. Una gran variació en un model el farà propens al soroll aleatori present al conjunt de dades, de manera que s'ajustarà massa al model.

Per comprendre amb més detall la compensació de biaix i variància en els models d'aprenentatge automàtic, podeu consultar aquest [article](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229). 

Un cop arribats a aquest punt, podem dividir els ensembles en quatre categories: 

1. **Bagging**: el bagging s'utilitza principalment per reduir la variació en un model. Un exemple simple de bagging és l'algorisme Random Forest.

2. **Boosting**: el boosting s'utilitza principalment per reduir el biaix en un model. Exemples d'algorismes d'impuls són Ada-Boost, XGBoost, arbres de decisió millorats per gradient, etc.

3. **Stacking**: el stacking s'utilitza principalment per augmentar la precisió de predicció d'un model. Per implementar l'stacking farem servir la biblioteca mlextend proporcionada per scikit learn.

4. **Cascading**: aquesta classe de models són molt precisos. La connexió en cascada s'usa principalment en escenaris on no es pot permetre cometre un error. Per exemple, una tècnica en cascada es fa servir principalment per detectar transaccions fraudulentes amb targetes de crèdit.

## Dades

Per a aquest exercici farem servir el dataset *diabetes.csv*. Aquest conjunt de dades és original de l'Institut Nacional de Diabetis i Malalties Digestives i Renals. L'objectiu d'aquest dataset és predir, basant-nos en els mesuraments de diagnòstic, si un pacient té diabetis.

En particular, tots els pacients són aquí dones d'almenys 21 anys, d'ascendència índia Pima.

El dataset conté la següent informació

- Embarassos: nombre d'embarassos
- Glucosa: concentració de glucosa en plasma a 2 hores en una prova de tolerància a la glucosa oral
- Pressió arterial: pressió arterial diastòlica (mm Hg)
- SkinThickness: gruix del plec cutani del tríceps (mm)
- Insulina: insulina sèrica de 2 hores (mu U / ml)
- IMC: índex de massa corporal (pes en kg / (alçada en m) ^ 2)
- DiabetesPedigreeFunction: funció del pedigrí de la diabetis
- Edat: edat (anys)
- Resultat (variable objectiu): variable de classe (0 o 1) 

A la primera part d'aquest exemple veurem la combinació de classificadors en paral·lel mitjançant les tècniques de **_Bagging_** i **_Boosting_**.

Per començar, vegem com és el dataset.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

diabetes = pd.read_csv("../data/diabetes.csv")

nRow, nCol = diabetes.shape
print(f"Hi ha {nRow} files i {nCol} columnes")
diabetes.tail()

Hi ha 768 files i 9 columnes


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


Per poder provar diversos models, primer dividirem el dataset entre _train_ i _test_.

Perquè tots obtingueu els mateixos resultats i poder comentar dubtes pel fòrum, fixarem la seed per obtenir els mateixos datasets de train i test.

Com en aquest exercici tractarem *stacking* i *cascading*, i tots dos s'apliquen sobre el conjunt de test, farem un *split* del 60 % per tenir una mica més de base en aplicar aquestes dues tècniques.

In [2]:
from sklearn.model_selection import train_test_split

myseed = 38
X = diabetes.drop(columns="Outcome")
y = diabetes["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.6, random_state=myseed
)

## Combinació paral·lela de classificadors

### 1. Arbres de decisió

Per poder comparar l'augment de  *performance* obtingut a mesura que anem aplicant tècniques noves, utilitzarem com  *baseline* un simple arbre de decisió.

A continuació, entrenarem un arbre de decisió sobre el conjunt de dades de train amb profunditat màxima de 3 nivells (aplicarem la mateixa restricció a les següents seccions), i avaluarem sobre test i calcularem la seva precisió aplicant validació creuada amb 5 conjunts.
   
<u>Més informació</u>: 
- [*cross_val_score*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
- [*cross validation*](http://scikit-learn.org/stable/modules/cross_validation.html)

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(max_depth=3, random_state=myseed)

cvscores = cross_val_score(clf, X_train, y_train, cv=5)
print(
    "Precisió mitjana obtinguda amb CV: {:.2f} +/- {:.2f} %".format(
        np.mean(cvscores) * 100, np.std(cvscores) * 100
    )
)

Precisió mitjana obtinguda amb CV: 71.04 +/- 4.82 %


In [4]:
from sklearn.metrics import accuracy_score

clf.fit(X_train, y_train)
y_pred_tree = clf.predict(X_test)
accuracy_score(y_pred_tree, y_test)

0.7462039045553145

Obtenim una precisió en test no massa alta, una mica per sota de la de train (això pot variar depenent dels datasets de train i test).

### 2. *Bagging*

La idea central del bagging és fer servir rèpliques del conjunt de dades original i fer-les servir per entrenar diferents classificadors.

Crearem subconjunts mostrant aleatòriament un munt de punts del conjunt de dades d'entrenament amb reemplaçament.

Ara entrenarem classificadors individuals a cadascun d'aquests subconjunts bootstrap. 

Cada un d'aquests classificadors base predirà l'etiqueta de classe per a un problema donat. Aquí és on combinem les prediccions de tots els models base. Aquesta part s'anomena etapa d'agregació. És per això que trobareu els ensembles bagging pel nom d'ensembles d'agregació. 

En general, es fa servir un vot de majoria simple en un sistema de classificació i es pren la mitjana de totes les prediccions per als models de regressió per combinar tots els classificadors base en un sol model i proporcionar el resultat final del model de conjunt.

Un exemple simple d'aquest enfocament és l'algorisme Random Forest. El bagging redueix l'alta variació (variança) d'un model, reduint així l'error de generalització. És un mètode molt eficaç, especialment quan tenim dades molt limitades com podria ser el nostre cas.

Mitjançant l'ús de mostres de bootstrap, podem obtenir una estimació afegint les puntuacions de moltes mostres.

**Com faríem el bagging?**

Suposem que tenim un conjunt d'entrenament que conté 100.000 punts de dades. 

Crearíem N subconjunts mostrejant a l'atzar 50K punts de dades per a cada subconjunt. 

Cadascun d'aquests N subconjunts s'utilitzarà per entrenar N classificadors diferents. 

A l'etapa d'agregació, totes aquestes N prediccions es combinaran en un sol model, també anomenat metaclassificador. 

Dels 100.000 punts presents originalment al conjunt de dades, si eliminem 1.000 punts, l'impacte que tindrà als conjunts de dades mostrades serà molt inferior.

Si pensem intuïtivament, és possible que alguns d'aquests 1.000 punts no siguin presents a tots els conjunts de dades mostrejades i, per tant, la quantitat de punts que s'eliminaran de cada conjunt de dades mostrejades serà molt inferior. Fins i tot zero en alguns casos! En resum, l'impacte d'eliminar 1.000 punts d'aquest tipus serà en realitat menor als classificadors base, cosa que reduirà la variació en un model i el farà més sòlid. 

La variància no és més que sensibilitat al soroll, com hem comentat anteriorment.

Seguidament, entrenarem un _**Random Forest**_ sobre el conjunt de dades de _train_ amb <b>20 arbres</b> de decisió i <b>profunditat màxima de 3</b> nivells, i avaluarem sobre test i calcularem la seva precisió aplicant una validació creuada amb 5 conjunts.

<u>Més informació</u>: 
- [*RandomForestClassifier*](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [5]:
from sklearn import ensemble

clf = ensemble.RandomForestClassifier(n_estimators=20, max_depth=3, random_state=myseed)

cvscores = cross_val_score(clf, X_train, y_train, cv=5)
print(
    "Precisió mitjana obtinguda amb CV: {:.2f} +/- {:.2f} %".format(
        np.mean(cvscores) * 100, np.std(cvscores) * 100
    )
)

Precisió mitjana obtinguda amb CV: 73.29 +/- 1.57 %


In [6]:
clf.fit(X_train, y_train)
y_pred_random_forest = clf.predict(X_test)
accuracy_score(y_pred_random_forest, y_test)

0.7418655097613883

Obtenim una bona precisió en test, una mica per sota de la de train. També notem una millora (encara que lleu) respecte a un simple arbre de decisió.

### 3. *Boosting*

El _boosting_ s'utilitza per convertir els classificadors de base feble en forts. Els classificadors febles generalment tenen una correlació molt feble amb les etiquetes de classe veritables i els classificadors forts tenen una correlació molt alta entre el model i les etiquetes de classe veritables.

El _boosting_ capacita els classificadors febles de manera iterativa, cadascun mirant de corregir l'error comès pel model anterior. Això s'aconsegueix entrenant un model feble en totes les dades d'entrenament, tot construint un segon model que té com a objectiu corregir els errors comesos pel primer model. Després construïm un tercer model que intenta corregir els errors comesos pel segon model i així successivament. Els models s'agreguen de manera iterativa fins que el model final ha corregit tots els errors comesos per tots els models anteriors.

Quan s'afegeixen els models a cada etapa, s'assignen alguns pesos al model relacionat amb la precisió del model anterior. Després d'afegir un classificador feble, els pesos es tornen a ajustar. Els punts classificats incorrectament reben pesos més alts i els punts classificats correctament reben pesos més baixos. Aquest enfocament farà que el següent classificador se centri en els errors comesos pel model anterior.

El boosting redueix l'error de generalització prenent un model d'alt biaix i baixa variància i reduint el biaix en un nivell significatiu. Recorda, el bagging redueix la variància. Com el bagging, el boosting també ens permet treballar amb models de classificació i regressió.

Entrenarem un _**Gradient Boosting**_ sobre el conjunt de dades de _train_ amb 20 arbres de decisió i profunditat màxima de 3 nivells. A continuació, avaluarem sobre test i calcularem la seva precisió aplicant una validació creuada amb 5 conjunts.

<u>Més informació</u>: 
- [*GradientBoostingClassifier*](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

In [7]:
clf = ensemble.GradientBoostingClassifier(
    n_estimators=20, max_depth=3, random_state=myseed
)

cvscores = cross_val_score(clf, X_train, y_train, cv=5)
print(
    "Precisió mitjana obtinguda amb CV: {:.2f} +/- {:.2f} %".format(
        np.mean(cvscores) * 100, np.std(cvscores) * 100
    )
)

Precisió mitjana obtinguda amb CV: 72.00 +/- 4.23 %


In [8]:
clf.fit(X_train, y_train)
y_pred_gradient_boosting = clf.predict(X_test)
accuracy_score(y_pred_gradient_boosting, y_test)

0.7613882863340564